<a href="https://colab.research.google.com/github/ysamokhotov/yandex-praktikum-projects/blob/main/SQL%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%D0%B0%D1%81%D1%81%D0%BE%D1%80%D1%82%D0%B8%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%20%D0%BA%D0%BD%D0%B8%D0%B6%D0%BD%D0%BE%D0%B3%D0%BE%20%D0%BC%D0%B0%D0%B3%D0%B0%D0%B7%D0%B8%D0%BD%D0%B0/SQL_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B0%D1%81%D1%81%D0%BE%D1%80%D1%82%D0%B8%D0%BC%D0%B5%D0%BD%D1%82%D0%B0_%D0%BA%D0%BD%D0%B8%D0%B6%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BC%D0%B0%D0%B3%D0%B0%D0%B7%D0%B8%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**План работы:**

1) [Импорт библиотек и подключение к серверу](#1)

2) [Исследуем таблицы - выведем первые несколько строк](#2)

3) [Посчитаем сколько книг вышло после 1 января 2000 года](#3)

4) [По каждой книге посчитаем количество обзоров и среднюю оценку](#4)

5) [Определим издательство, которое выпустило наибольшее число книг толще 50 страниц для исключения брошюр](#5)

6) [Определим автора с самой высокой средней оценкой книг (для книг с 50 и более оценками)](#6)

7) [Посчитаем среднее количество обзоров от пользователей (поставивших больше 50 оценок)](#7)

# Импорт библиотек и подключение к серверу<a id=1></a>

In [8]:
import pandas as pd
from sqlalchemy import create_engine

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
db_config = {'user': 'praktikum_student',
    'pwd': 'Sdf4$2;d-d30pp',
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,
    'db': 'data-analyst-final-project-db'}

In [11]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']) 

In [12]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Исследуем таблицы - выведем первые несколько строк<a id=2></a>

In [13]:
def select(query):
    return pd.io.sql.read_sql(query, con = engine)

In [14]:
select(
    '''
    SELECT *
    FROM books
    LIMIT 5
    '''
)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [15]:
select(
    '''
    SELECT *
    FROM authors
    LIMIT 5
    '''
)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [16]:
select(
    '''
    SELECT *
    FROM ratings
    LIMIT 5
    '''
)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [17]:
select(
    '''
    SELECT *
    FROM reviews
    LIMIT 5
    '''
)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [18]:
select(
    '''
    SELECT *
    FROM publishers
    LIMIT 5
    '''
)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


# Посчитаем сколько книг вышло после 1 января 2000 года<a id=3></a>

In [19]:
select(
    '''
    SELECT COUNT(book_id)
    FROM books
    WHERE publication_date::DATE >= '2000-01-01'
    '''
)

,count
0,821


*С 1 января 2020 вышла 821 книга.*

# По каждой книге посчитаем количество обзоров и среднюю оценку<a id=4></a>

In [20]:
select(
    '''
    WITH t AS
    (SELECT book_id, COUNT(review_id) review_count
    FROM reviews
    GROUP BY book_id), 

    tt AS
    (SELECT book_id, AVG(rating) avg_rating
    FROM ratings
    GROUP BY book_id)

    SELECT b.title, review_count, avg_rating
    FROM books b
    LEFT JOIN t ON b.book_id = t.book_id
    LEFT JOIN tt ON b.book_id = tt.book_id
    WHERE review_count > 5
    ORDER BY avg_rating DESC
    LIMIT 5
    '''
)

,title,review_count,avg_rating
0,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
1,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
2,The Book Thief,6,4.264151
3,The Glass Castle,6,4.206897
4,The Hobbit or There and Back Again,6,4.125000


*По всем книгам проставлен средний рейтинг и количество отзывов.*

# Определим издательство, которое выпустило наибольшее число книг толще 50 страниц для исключения брошюр<a id=5></a>

In [21]:
select(
    '''
    SELECT publisher, COUNT(book_id)
    FROM books b
    LEFT JOIN publishers p ON p.publisher_id = b.publisher_id
    WHERE num_pages > 50
    GROUP BY publisher
    ORDER BY count DESC
    LIMIT 1
    '''
)

,publisher,count
0,Penguin Books,42


*Penguin Books выпустили больше всего многостраничных книг - 42.*

# Определим автора с самой высокой средней оценкой книг (для книг с 50 и более оценками)<a id=6></a>

In [22]:
select(
    '''
    WITH t AS
    (SELECT book_id, AVG(rating)
    FROM ratings
    GROUP BY book_id)

    SELECT author, AVG(avg) avg_rating
    FROM
        (SELECT *
        FROM books
        WHERE book_id IN(
            SELECT book_id
            FROM
                (SELECT book_id, COUNT(rating_id) ratings_quantity
                FROM ratings
                GROUP BY book_id) t
                WHERE ratings_quantity >= 50)) tt
    LEFT JOIN t ON t.book_id = tt.book_id
    LEFT JOIN authors a ON a.author_id = tt.author_id
    GROUP BY author
    ORDER BY avg_rating DESC
    LIMIT 1
    '''
)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


*Самый высокий рейтинг среди авторов книг с 50 и более оценками - Джоан Роулинг.*

# Посчитаем среднее количество обзоров от пользователей (поставивших больше 50 оценок)<a id=7></a>

In [24]:
select(
    '''
    SELECT ROUND(AVG(reviews_quantity))
    FROM
        (SELECT username, COUNT(review_id) reviews_quantity
        FROM reviews
        WHERE username IN(
            SELECT username
            FROM
                (SELECT username, COUNT(rating_id) ratings_quantity
                FROM ratings
                GROUP BY username) t
            WHERE ratings_quantity > 50)
        GROUP BY username) tt
    '''
)

,round
0,24.0


*В среднем пользователи, оставившие более 50 оценок, оставляют 24 отзыва.*

**Выводы:** всего в библиотеке представлено 1000 книг, из них 821 вышла с 2020 года. По всем книгам в базе есть рейтинг и хотя бы один отзыв. Больше всего многостраничных книг выпустило издание Penguin Books. Среди книг с >= 50 оценками нивысший средний рейтинг у Джоан Роулинг. В среднем пользователи, поставившие > 50 оценок, оставляют 24 отзыва.